In [19]:
#conda activate AP1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from scipy.stats import skew, kurtosis

import pyfolio as pf
import empyrical as emp

In [20]:
df = pd.read_excel('data_nn.xlsx')
#df.to_pickle("data_nn.xlsx")

In [21]:
# Set the first column as the date index
df.set_index(df.columns[0], inplace=True)

# Convert the index to string and then to DatetimeIndex format
df.index = pd.to_datetime(df.index.astype(str))

# Filter the data for the last ten years
df_last_10_years = df.loc[df.index > "2020-01-02"]

# Apply rolling sum with a window of 252 and require at least 126 non-NaN values
df_rolling_sum = df_last_10_years.rolling(window=252, min_periods=int(252//2)).sum()

# Forward-fill NaN values, but limit this to a maximum of 5 consecutive fills
df_filled = df_last_10_years.ffill(limit=5)

# Drop any remaining NaN values that still exist after the forward-fill operation
df_cleaned = df_filled.dropna()

#return back original name to not interruppt code.
df_last_10_years = df_cleaned




In [75]:
def refactored_advanced_features(df_returns):
    """
    Refactored computation of advanced financial features to reduce DataFrame fragmentation.
    """
    skew = {}
    kurtosis = {}
    max_drawdown = {}
    volatility = {}
    vaR = {}
    momentum = {}
    avg_return = {}
    rsi = {}

        
        # 1. Skewness
    print("Skewness")
    for window in [20, 40, 60, 100, 180, 240, 360, 480]:
        skew[window] = df_returns.rolling(window).skew()

        # 2. Kurtosis
    print("Kurtosis")
    for window in [20, 40, 60, 100, 180, 240, 360, 480]:
        kurtosis[window]=df_returns.rolling(window).kurt()
    
    # 3. Maximum drawdown
    print("Maximum drawdown")
    for window in [20, 40, 60, 100, 180, 240, 360, 480]:
        max_drawdown[window] = df_returns.rolling(window).apply(emp.max_drawdown, raw=True)
    
    # 4. Volatility
    print("Volatility")
    for window in [20, 40, 60, 100, 180, 240, 360, 480]:
        volatility[window] = df_returns.rolling(window).std()*(252**0.5)
    
    # 5. Value at Risk
    print("Value at Risk")
    for window in [20, 40, 60, 100, 180, 240, 360, 480]:
        vaR[window] = df_returns.rolling(window).apply(emp.value_at_risk, raw=True)
    
    # 6. Momentum
    print("Momentum")
    for window in [20, 40, 60, 100, 180, 240, 360, 480]:
        momentum[window] = df_returns.rolling(window).sum() # ?

    print("Average Return")
    for window in [20, 40, 60, 100, 180, 240, 360, 480]:
        avg_return[window] = df_returns.rolling(window).mean()
    
    return skew, kurtosis, max_drawdown, volatility, vaR, momentum, avg_return

# This function reduces DataFrame fragmentation by constructing all columns and concatenating them at once.

# Läs tommys mex hur de gjorde reversal, sen implementera det. Fixa windows size till vad de hade i rapporten.
# skew[20].head() 

In [ ]:
# Call the function and capture the output
skew, kurtosis, max_drawdown, volatility, vaR, momentum, avg_return = refactored_advanced_features(df_last_10_years)


In [ ]:
# Reset the feature DataFrames list
features_df_list = []

# Create individual lists for each feature's DataFrame
skew_df_list = [] 
kurtosis_df_list = []
max_drawdown_df_list = []
volatility_df_list = []
vaR_df_list = []
momentum_df_list = []
avg_return_df_list = []

# Windows configuration
windows = [20, 40, 60, 100, 180, 240, 360, 480]

# Iterate through each feature dictionary and create a DataFrame
for feature_name, feature_dict in [('skew', skew), ('kurtosis', kurtosis), ('max_drawdown', max_drawdown), 
                                   ('volatility', volatility), ('vaR', vaR), ('momentum', momentum), ('avg_return', avg_return)]:
    # Only keep the windows that are present for each feature
    relevant_windows = windows if feature_name != 'kurtosis' else windows[:-1]
    feature_df = pd.concat({f'{feature_name}_{window}': feature_dict[window] for window in relevant_windows}, axis=1)
    
    # Append the individual DataFrame to the corresponding feature list
    if feature_name == 'skew':
        skew_df_list.append(feature_df)
    elif feature_name == 'kurtosis':
        kurtosis_df_list.append(feature_df)
    elif feature_name == 'max_drawdown':
        max_drawdown_df_list.append(feature_df)
    elif feature_name == 'volatility':
        volatility_df_list.append(feature_df)
    elif feature_name == 'vaR':
        vaR_df_list.append(feature_df)
    elif feature_name == 'momentum':
        momentum_df_list.append(feature_df)
    elif feature_name == 'avg_return':
        avg_return_df_list.append(feature_df)
    
    # Add the DataFrame to the main list
    features_df_list.append(feature_df)

# Concatenate all feature DataFrames into a single DataFrame
features_df = pd.concat(features_df_list, axis=1)

# The individual lists for each feature now contain their respective DataFrames
# And features_df_list contains all the feature DataFrames
# Let's print the first item of each sublist to confirm
#print("Skew DataFrame:\n", skew_df_list[0].tail(), "\n")
#print("Kurtosis DataFrame:\n", kurtosis_df_list[0].tail(), "\n")
#print("Max Drawdown DataFrame:\n", max_drawdown_df_list[0].tail(), "\n")
#print("Volatility DataFrame:\n", volatility_df_list[0].tail(), "\n")
#print("VaR DataFrame:\n", vaR_df_list[0].tail(), "\n")
#print("Momentum DataFrame:\n", momentum_df_list[0].tail(), "\n")
#print("Average Return DataFrame:\n", avg_return_df_list[0].tail(), "\n")

# Print the last 5 rows of the combined DataFrame
features_df.tail()

In [59]:
def RSI(df_returns, window):
    """
    Computes the Relative Strength Index (RSI) for a given window.
    """
    df = df_returns.copy()
    df[df >= 0] = 1
    df[df < 0] = 0
    df = df.rolling(window).mean()*100
    return df

RSI skip for now

In [ ]:
# Initialize an empty dictionary to store the last RSI value for each window
rsi_values = {}

# Calculate RSI for each window and store the last value
for window in [20, 40, 60, 100, 180, 240, 360, 480]:
    rsi_df = RSI(df_last_10_years, window)  # df_returns is your DataFrame with returns data
    last_rsi_value = rsi_df.iloc[-1]  # Get the last row of the RSI DataFrame
    rsi_values[window] = last_rsi_value  # Store it in the dictionary with the window as the key

# Print the last RSI value for a 20-day window
print("Last RSI value for 20-day window:")
print(rsi_values[20])




NN

In [102]:
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
# Define the neural network model to accommodate multiple output dimensions
class MultivariateNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MultivariateNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, output_dim)  # Output dimension should match the number of columns
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)


# Using actual columns from the dataset to populate 'dataframes_to_concat'
# As per the uploaded dataset, 'Equities_0', 'Equities_1', and 'Equities_2' are used
# Assume that calculated_features_df is a DataFrame containing all the calculated features
#calculated_features = skew, kurtosis, max_drawdown, volatility, vaR, momentum, avg_return
calculated_features = feature_df
calculated_features_df = pd.DataFrame(calculated_features)
dataframes_to_concat = [calculated_features_df]

# Concatenate feature data frames 
features_df = pd.concat(dataframes_to_concat, axis=1)
features_df.fillna(features_df.mean(), inplace=True)


# Split the data into training and testing sets (considering all columns for y)
X_train, X_test, y_train, y_test = train_test_split(features_df, features_df, test_size=0.2, random_state=42)

# Scaling the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(f"Debug: NaN values in training data: {np.isnan(X_train_scaled).sum()}")

# Converting to PyTorch tensor
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train.values)  # Multiple columns are included
X_test_tensor = torch.FloatTensor(X_test_scaled)

# Initialize the neural network model
output_dim = y_train.shape[1]  # Number of columns to predict
model = MultivariateNN(X_train.shape[1], output_dim)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
epochs = 50
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

initial_weights = model.fc1.weight.data.numpy()
print(f"Debug: Initial weights are {initial_weights}")

# Added check for empty list before DataFrame concatenation

# Check if dataframes_to_concat is empty before attempting concatenation
if len(dataframes_to_concat) == 0:
    print("Warning: No DataFrames to concatenate.")
else:
    features_df = pd.concat(dataframes_to_concat, axis=1)

# Placeholder for Backward Neural Network
class BackwardNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BackwardNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)
        
# # Initialize the backward neural network model
# backward_model = BackwardNN(X_train.shape[1], output_dim)

# # Training loop for backward neural network
# for epoch in range(epochs):
#     backward_model.train()
#     optimizer.zero_grad()
#     backward_outputs = backward_model(X_train_tensor)
#     backward_loss = criterion(backward_outputs, y_train_tensor)
#     backward_loss.backward()
#     optimizer.step()

# After training
first_layer_weights = model.fc1.weight.data.numpy()
feature_importance = np.abs(first_layer_weights).sum(axis=0)
first_layer_weights = model.fc1.weight.data.numpy()
if first_layer_weights.size == 0:
    print("Debug: The first layer weights are empty.")
else:
    print(f"Debug: The first layer weights have a shape of {first_layer_weights.shape}.")

# Debug Statement  Calculate and display feature importance
feature_importance = np.abs(first_layer_weights).sum(axis=0)
if feature_importance.size == 0:
    print("Debug: Feature importance calculation resulted in an empty array.")
else:
    print(f"Debug: Feature importance is {feature_importance}.")



Debug: NaN values in training data: 0
Debug: Initial weights are [[ 0.01579001  0.04624189  0.0554379  ... -0.0016118  -0.0041359
  -0.04692262]
 [-0.05253757 -0.03971265 -0.05537774 ...  0.06627474  0.0211127
   0.05498072]
 [-0.05942423  0.00161982 -0.06776381 ... -0.04199161 -0.01984524
  -0.05122317]
 ...
 [ 0.02323836  0.01368109  0.01894289 ...  0.04954439  0.00701638
   0.07517029]
 [-0.08546984 -0.03160863 -0.07787771 ...  0.00322067  0.00729904
  -0.00902276]
 [-0.069322   -0.04149174 -0.07131457 ... -0.07467739 -0.07709451
  -0.04498061]]
Debug: The first layer weights have a shape of (128, 448).
Debug: Feature importance is [6.7518516 6.179517  6.3350725 7.079707  6.2820826 6.712406  6.8396134
 7.280704  6.80403   6.4681373 6.6247606 7.0132575 6.8663735 6.9801826
 6.755113  7.1682596 6.807978  6.890221  6.580156  6.806979  6.434308
 6.44356   6.305599  6.3985233 6.993012  6.864393  7.3122306 7.2732897
 6.7891855 6.651632  7.3493924 7.2543397 6.2830815 7.002481  6.634133
 7.3

Risk budgeting benchmark

In [ ]:
import cvxpy as cp
from tqdm import tqdm


# Assuming df_last_10_years contains the daily returns
returns = df_last_10_years.drop(columns=df_last_10_years.columns[0])

# Initialize list to store optimal weights and returns for each t
all_weights = []
all_returns = []

# Number of assets
n_assets = len(returns.columns)

# Risk budget for each asset
b = np.ones(n_assets) / n_assets  # For example, equal risk budgeting

# Arbitrary constant for the constraint
c = 1  # Example value, adjust as needed

# Start from the 21st observation
for t in tqdm(range(54,len(returns) - 1,5)):
    # Data up to time t
    data_t = returns.iloc[:t+1].astype("float16")
    print(data_t.index[-1])
    # Covariance matrix of the returns
    cov_matrix_values = data_t.cov().values
    cov_matrix_values = (cov_matrix_values + cov_matrix_values.T)/2

    #cov_matrix_values += np.eye(cov_matrix_values.shape[0],cov_matrix_values.shape[1])

    #if min(eigs) <= 0:
    #    print(t)
    # Portfolio weights as a variable (y)
    y = cp.Variable(shape=n_assets)
    # Objective function: Minimize the square root of the portfolio variance
    objective = cp.Minimize(cp.sqrt(cp.quad_form(y, cp.psd_wrap(cov_matrix_values))))

    # Alternative solutions

    #objective = cp.Minimize(cp.quad_form(y, cp.psd_wrap(cov_matrix_values)))

    #cov_matrix_values = np.array(cov_matrix_values)
    #cov_matrix_values += np.eye(cov_matrix_values.shape[0],cov_matrix_values.shape[1])
    #L = np.linalg.cholesky(cov_matrix_values)
    #objective = cp.Minimize(cp.norm(L@y,2))

    # Constraints:
    # 1. The weights must sum to 1 (full investment)
    # 2. The risk budgeting constraint must be satisfied
    # 3. The weights must be non-negative
    constraints = [
        #cp.sum(y) == 1,        # incompatible with contstraint below. Normalize afterwards
        cp.sum(cp.multiply(b, cp.log(y))) >= c,
        y >= 1e-5 #strict inequalities are not allowed
    ]

    # Formulate the optimization problem
    problem = cp.Problem(objective, constraints)

    # Solve the problem using a suitable solver
    problem.solve(solver=cp.SCS,qcp=True, eps = 1e-5, max_iters  = 100) #kan ange hur många försöja man vill att solve kan göra. Kolla upp det. "Maxiteration". googla på risk budgeting med cvxpy.
    # Store the optimal weights for time t
    optimal_weights = y.value
    #optimal_weights /= np.linalg.norm(optimal_weights,1)
    all_weights.append(optimal_weights)

    #print(optimal_weights)

    #print(type(all_weights))
    #print(type(returns.iloc[t+1].values))
    
    # Calculate and store the portfolio return for time t+1 using weights from time t
    next_return = np.dot(returns.iloc[t+1].values, optimal_weights)
    all_returns.append(next_return)

# Convert lists to arrays for further analysis if needed
#all_weights = np.array(all_weights)
#all_returns = np.array(all_returns)

# Print the first 5 values of all_returns
#print("First 5 values of all_returns:")
#print(all_returns[:5])